In [1]:
import pandas as pd # 引用套件並縮寫為 pd  
import numpy as np
import re
import string
import tqdm

In [2]:
num_top_word=1000
maxlen=30

In [3]:
data_train = pd.read_csv('data/train.csv')  
print(data_train) 

         id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7608  10869     NaN      NaN   
7609  10870     NaN      NaN   
7610  10871     NaN      NaN   
7611  10872     NaN      NaN   
7612  10873     NaN      NaN   

                                                   text  target  
0     Our Deeds are the Reason of this #earthquake M...       1  
1                Forest fire near La Ronge Sask. Canada       1  
2     All residents asked to 'shelter in place' are ...       1  
3     13,000 people receive #wildfires evacuation or...       1  
4     Just got sent this photo from Ruby #Alaska as ...       1  
...                                                 ...     ...  
7608  Two giant cranes holding a bridge collapse int...       1  
7609  @aria_ahrary @TheTawniest The out of control w...       1  
7610  M1.94 [01:04 UT

In [4]:
data_test = pd.read_csv('data/test.csv')  
print(data_test) 

         id keyword location  \
0         0     NaN      NaN   
1         2     NaN      NaN   
2         3     NaN      NaN   
3         9     NaN      NaN   
4        11     NaN      NaN   
...     ...     ...      ...   
3258  10861     NaN      NaN   
3259  10865     NaN      NaN   
3260  10868     NaN      NaN   
3261  10874     NaN      NaN   
3262  10875     NaN      NaN   

                                                   text  
0                    Just happened a terrible car crash  
1     Heard about #earthquake is different cities, s...  
2     there is a forest fire at spot pond, geese are...  
3              Apocalypse lighting. #Spokane #wildfires  
4         Typhoon Soudelor kills 28 in China and Taiwan  
...                                                 ...  
3258  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...  
3259  Storm in RI worse than last hurricane. My city...  
3260  Green Line derailment in Chicago http://t.co/U...  
3261  MEG issues Hazardous Weather 

In [5]:
AllData=pd.concat([data_train,data_test])
AllData
#AllData[:7613]為train data; AllData[7613:]為test data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1.0
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,NaN
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,NaN
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,NaN
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,NaN


In [6]:
AllData=AllData[['text','target']]
AllData['text']=AllData['text'].str.lower()
AllData

C:\Users\highs\anaconda3\envs\tensorflow-gpu 36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,text,target
0,our deeds are the reason of this #earthquake m...,1.0
1,forest fire near la ronge sask. canada,1.0
2,all residents asked to 'shelter in place' are ...,1.0
3,"13,000 people receive #wildfires evacuation or...",1.0
4,just got sent this photo from ruby #alaska as ...,1.0
...,...,...
3258,earthquake safety los angeles ûò safety faste...,NaN
3259,storm in ri worse than last hurricane. my city...,NaN
3260,green line derailment in chicago http://t.co/u...,NaN
3261,meg issues hazardous weather outlook (hwo) htt...,NaN


# Data Clean

In [7]:
#remove url
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

#Removing HTML tags
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

# Romoving Emojis
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Removing punctuations
def remove_punct(text):
    table=str.maketrans('','',string.punctuation)
    return text.translate(table)



In [8]:
#AllData['text']=AllData['text'].apply(lambda x : remove_URL(x))
#AllData['text']=AllData['text'].apply(lambda x : remove_html(x))
#AllData['text']=AllData['text'].apply(lambda x: remove_emoji(x))
AllData['text']=AllData['text'].apply(lambda x : remove_punct(x))

C:\Users\highs\anaconda3\envs\tensorflow-gpu 36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


# Data Encoding

In [9]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [10]:
AllData_=AllData['text']

In [11]:
tokenizer=Tokenizer(num_words=num_top_word)
tokenizer.fit_on_texts(AllData_)
seq=tokenizer.texts_to_sequences(AllData_)
seq

[[100, 20, 1, 812, 5, 18, 236, 136, 67, 39],
 [154, 41, 200, 813],
 [39, 3, 4, 715, 20, 130, 16, 36, 283, 260, 53, 4, 715, 20],
 [57, 260, 4, 98],
 [27, 95, 18, 284, 19, 25, 269, 19, 66, 2, 198],
 [418, 98, 649, 861, 4, 770, 398, 3, 389, 41],
 [199, 74, 895, 309, 862, 285, 5, 4],
 [34, 10, 211, 5, 1, 6, 7, 70, 102, 2, 41, 4, 1],
 [333, 42, 71, 260, 52, 4, 1, 549, 863, 1, 657],
 [34, 15, 1, 591, 8, 237, 3, 100, 455],
 [501, 57, 502, 19, 1, 270, 419, 32, 550],
 [896,
  616,
  8,
  224,
  677,
  2,
  678,
  7,
  189,
  4,
  616,
  58,
  175,
  7,
  261,
  65,
  58,
  175,
  7,
  261,
  65,
  285],
 [285, 53, 617, 300, 814],
 [199, 4, 43],
 [247, 3, 198, 456, 10, 4, 123, 85, 352],
 [637, 26, 105],
 [7, 106],
 [254, 8],
 [12, 123, 8, 32, 842],
 [58, 2],
 [18, 8],
 [8, 618],
 [106],
 [58, 2, 107],
 [],
 [36, 110, 15, 212],
 [22, 4, 125, 353],
 [106, 12],
 [],
 [65, 11, 24],
 [1, 378],
 [592],
 [43, 262, 658, 3, 843, 1, 895, 90],
 [352, 815, 274, 592, 4],
 [35, 9, 50, 274, 28, 592],
 [10, 755

In [12]:
seq = np.array(seq)

C:\Users\highs\anaconda3\envs\tensorflow-gpu 36\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [13]:
from keras.preprocessing import sequence
seq=sequence.pad_sequences(seq, maxlen=maxlen)

In [14]:
seq.shape

(10876, 30)

### 使用 pre-train Embedding model Glove 

In [ ]:
embedding_dict={}
with open('data/glove.6B.100d.txt','r',encoding="utf-8") as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()
embedding_dict

In [ ]:
word_index=tokenizer.word_index

embedding_matrix=np.zeros((num_top_word+1,100))

for word,i in (word_index.items()):
    if i > num_top_word:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

### check Glove 覆蓋率

In [ ]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in (vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [ ]:
sentences = AllData["text"].apply(lambda x: x.split()).values

In [ ]:
sentences

In [ ]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in (sentences):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
vocab = build_vocab(sentences)

In [ ]:
oov = check_coverage(vocab,word_index)

In [ ]:
oov[:20]

## 建立model

In [ ]:
from keras import layers
from keras import models
from keras import regularizers
from keras.layers import Embedding,LSTM,Dense,SimpleRNN
from keras.initializers import Constant

def create_model():
    model=models.Sequential()
    model.add(Embedding(num_top_word+1,100))
    model.add(layers.Bidirectional(LSTM(100)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [ ]:
model=create_model()
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable=False
model.summary()

In [ ]:
train_x=seq[:7613]
train_y=AllData['target'][:7613]

In [ ]:
train_x.shape

In [ ]:
history=model.fit(train_x,train_y,epochs=30,batch_size=128,validation_split=0.2)

In [ ]:
SSS

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
#plt.show()
plt.savefig("model_V5-1.png")

In [ ]:
plt.clf()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
#plt.show()
plt.savefig("model_V5-2.png")

In [ ]:
print('平均為:',sum(val_acc)/len(val_acc))

In [ ]:
val_acc.sort()
print('前五平均為:',sum(val_acc[-5:])/len(val_acc[-5:]))

In [ ]:
sss

# Retraining 

In [ ]:
#model1=create_model()
#history=model1.fit(seq,data_train['target'],epochs=6,batch_size=128)
#model.save('model_V5.h5')

In [ ]:
model.save('model_V5.h5')

# Fitting

In [ ]:
test_x=seq[7613:]
test_x

In [ ]:
res=model.predict(test_x)

In [ ]:
res[res>0.5]=1
res[res<=0.5]=0
res

In [ ]:
data_test['target']=res

In [ ]:
data_test

In [ ]:
res=data_test[['id','target']]
res

In [ ]:
res['target']=pd.to_numeric(res['target'], downcast='integer')
res.to_csv('Kaggle-5.csv',index=False)